# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [1]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

La clave de API parece buena


In [3]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [4]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

Título de la Web:
Frogames
Contenido de la Web:
Ir al contenido principal
Frogames
Menú alternativo
Menú
Iniciar sesión
Ganadora del premio 'Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023'
Pasión por
aprender
con los
mejores
En Frogames Formación te ayudamos a convertirte en todo un experto en: Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps, Data Science y mucho más.
Aprende mientras te diviertes
Cursos, Rutas y Suscripciones
Certificados de finalización
Qué encontrarás
dentro
de Frogames
Cursos online y formación de calidad para toda la família
Rutas temáticas
Rutas organizadas para que aprendas paso a paso, subiendo cada escalón e incrementando tus conocimientos adquiridos
Instructores Expertos
Con un equipo de profesionales y expertos en la materia que te acompañará a lo largo de todo el aprendizaje en la plataforma
Certificados blockchain
Títulos verificados por blockchain para cada habilidad que aprenda

['#main-content',
 '/',
 '/',
 '/users/sign_in',
 'https://cursos.frogamesformacion.com/pages/rutas',
 'https://cursos.frogamesformacion.com/pages/instructores',
 'https://cursos.frogamesformacion.com/pages/certificaciones',
 'https://cursos.frogamesformacion.com/collections',
 '/courses/bandas-sonoras-3',
 '/courses/bandas-sonoras-3',
 '/courses/fundamentos-teoria-musical',
 '/courses/fundamentos-teoria-musical',
 '/courses/bandas-sonoras-2',
 '/courses/bandas-sonoras-2',
 '/courses/liderazgo-data-drive',
 '/courses/liderazgo-data-drive',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/bandas-sonoras-peliculas-1',
 '/courses/ingenieria-llm',
 '/courses/ingenieria-llm',
 '/courses/principios-solid',
 '/courses/principios-solid',
 '/courses/matematicas-ml-2',
 '/courses/matematicas-ml-2',
 '/courses/obsidian',
 '/courses/obsidian',
 '/courses/trigonometria-desde-cero',
 '/courses/trigonometria-desde-cero',
 '/courses/kotlin-desde-cero',
 '/courses/kotlin-desde-cero',
 '/courses/matem

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [5]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [6]:
print(link_system_prompt)

Se te proporciona una lista de enlaces que se encuentran en una página web. Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.
Debes responder en JSON como en este ejemplo:
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(frog))

Aquí hay una lista de enlaces de la página web https://cursos.frogamesformacion.com - Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.
Links (puede que algunos sean links relativos):
#main-content
/
/
/users/sign_in
https://cursos.frogamesformacion.com/pages/rutas
https://cursos.frogamesformacion.com/pages/instructores
https://cursos.frogamesformacion.com/pages/certificaciones
https://cursos.frogamesformacion.com/collections
/courses/bandas-sonoras-3
/courses/bandas-sonoras-3
/courses/fundamentos-teoria-musical
/courses/fundamentos-teoria-musical
/courses/bandas-sonoras-2
/courses/bandas-sonoras-2
/courses/liderazgo-data-drive
/courses/liderazgo-data-drive
/courses/bandas-sonoras-peliculas-1
/courses/bandas-sonoras-peliculas-1
/courses/ingenieria-llm
/courses/ingenieria-llm
/courses/principios-solid
/courses/p

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://claude.ai/',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.an

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'Pagina Acerca de',
   'url': 'https://www.anthropic.com/company'},
  {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'Pagina de Investigación',
   'url': 'https://www.anthropic.com/research'},
  {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'},
  {'type': 'API', 'url': 'https://www.anthropic.com/api'},
  {'type': 'Pagina de precios', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'Pagina de Noticias', 'url': 'https://www.anthropic.com/news'}]}

In [12]:
get_links("https://cursos.frogamesformacion.com")

{'links': [{'type': 'Pagina Sobre nosotros',
   'url': 'https://cursos.frogamesformacion.com/pages/rutas'},
  {'type': 'Pagina de Instructores',
   'url': 'https://cursos.frogamesformacion.com/pages/instructores'},
  {'type': 'Pagina de Certificaciones',
   'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'},
  {'type': 'Pagina de Premios',
   'url': 'https://cursos.frogamesformacion.com/pages/premios'},
  {'type': 'Pagina para Empresas',
   'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'},
  {'type': 'Pagina de Carreras',
   'url': 'https://cursos.frogamesformacion.com/pages/afiliados'},
  {'type': 'Pagina de Cursos',
   'url': 'https://cursos.frogamesformacion.com/collections'},
  {'type': 'Pagina de Cursos',
   'url': 'https://cursos.frogamesformacion.com/courses/bandas-sonoras-3'}]}

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://anthropic.com"))

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de Precios', 'url': 'https://www.anthropic.com/pricing'}]}
Landing page:
Título de la Web:
Home \ Anthropic
Contenido de la Web:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22,

In [15]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [17]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de la empresa', 'url': 'https://www.anthropic.com'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de precios', 'url': 'https://www.anthropic.com/pricing'}]}


'Estás mirando una empresa llamada: Anthropic\nAquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\nLanding page:\nTítulo de la Web:\nHome \\ Anthropic\nContenido de la Web:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, Wh

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://www.anthropic.com/company'}, {'type': 'Pagina de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Pagina de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Pagina de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Pagina de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Pagina Claude', 'url': 'https://www.anthropic.com/claude'}]}


# Folleto de Anthropic

## Sobre Anthropic

Anthropic es una empresa de investigación y seguridad en IA con sede en San Francisco, centrada en el desarrollo de sistemas de inteligencia artificial que son fiables, interpretables y dirigibles. Nuestra misión es crear IA que beneficie a las personas y la sociedad, abordando tanto sus oportunidades como sus riesgos.

## Nuestra Visión y Cultura

En Anthropic, mantenemos que la **seguridad es una ciencia**. Tratamos la seguridad de la IA como un campo de investigación sistemática, apoyándonos en diversas técnicas para asegurar que los sistemas que desarrollamos son seguros. Fomentamos un ambiente de **alta confianza**, donde se privilegian la honestidad y la colaboración. Nuestros valores guían nuestras acciones:

1. **Aquí por la misión**: Nos enfocamos en garantizar que la IA transformadora sea una fuerza positiva para la humanidad.
2. **Confianza inusual**: Mantenemos un entorno de alta confianza y apertura intelectual.
3. **Un gran equipo**: La colaboración es esencial, y cada miembro aporta su experiencia única hacia nuestra misión compartida.
4. **Hacer lo simple que funciona**: Priorizamos enfoques pragmáticos y empíricos.

## Nuestros Clientes

Anthropic trabaja con diversas industrias, colaborando con empresas, organizaciones sin fines de lucro, e instituciones gubernamentales para fomentar la seguridad en el desarrollo de la IA. Algunos de nuestros productos son ampliamente utilizados por organizaciones que buscan crear soluciones potentes y seguras.

## Carreras en Anthropic

Estamos en búsqueda de individuos apasionados por la IA y su impacto en el mundo. Nutrimos un entorno inclusivo y apoyamos el crecimiento profesional a través de beneficios significativos:
- **Salud y Bienestar**: Seguro médico completo, beneficios de fertilidad inclusivos, y apoyo a la salud mental.
- **Compensación y Soporte**: Salarios competitivos, planes de jubilación, y equidad en acciones.
- **Beneficios adicionales**: Estipendios para educación, mudanza, y oficina en casa.

### ¿Por qué unirte a nosotros?

- Abrazamos la diversidad en la experiencia y conocimiento, por lo que tú, independientemente de tu trayectoria previa en IA, podemos estar interesados en ti.
- Nos dedicamos a fomentar un ambiente donde todos contribuyen y pueden tener un impacto real.

## Oportunidades de Aprendizaje

Anthropic ofrece recursos para aquellos que buscan prepararse para futuros roles en IA. Las oportunidades de desarrollo incluyen:
- Acceso a documentación técnica y guías de programación en nuestras plataformas, incluyendo el uso de modelos de Claude para el desarrollo de herramientas y aplicaciones.

## Contáctanos

Para más información sobre nuestras oportunidades laborales, productos o alianzas estratégicas, por favor visita [Anthropic](https://www.anthropic.com) o sigue nuestras redes sociales en Twitter, LinkedIn y YouTube.

---

En Anthropic, te invitamos a ser parte de la misión de construir un futuro seguro y beneficioso con IA. ¡Te esperamos!

In [20]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Página de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Página de Cursos', 'url': 'https://cursos.frogamesformacion.com/collections'}, {'type': 'Página de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Página de Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Página para Empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Carreras/Empleos', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}]}


# Folleto Informativo de Frogames Formación

## Acerca de Frogames Formación
Frogames Formación es una plataforma educativa galardonada con el premio a la "Enseñanza online de datos y competencias digitales más innovadora de Europa, 2023". Nos enfocamos en ofrecer formación de alta calidad en áreas como la Programación de Videojuegos, Inteligencia Artificial, Machine Learning, Desarrollo de Apps y Data Science. Desde nuestra fundación, más de 500.000 estudiantes han confiado en nosotros para desarrollar sus habilidades.

## Nuestra Cultura
En Frogames, fomentamos un ambiente de aprendizaje activo y divertido. Creemos en el valor de aprender mientras se disfruta y en la importancia de la comunidad. Nuestros instructores son expertos en sus campos, guían a los estudiantes de manera comprometida y están dedicados a hacer de la educación una experiencia gratificante.

## Lo que Ofrecemos
- **Cursos Online**: Amplia gama de cursos temáticos organizados para aprender de manera progresiva.
- **Certificados de Finalización**: Cada curso finalizado proporciona un certificado validado por tecnología blockchain, asegurando su autenticidad.
- **Rutas de Aprendizaje**: Aprendizaje estructurado que permite a los estudiantes adquirir o mejorar habilidades de manera escalonada.
- **Suscripciones**: Acceso ilimitado a nuestro catálogo de cursos actualizado semanalmente.

### Cursos Destacados
- **Desarrollo de Videojuegos con Unity**: Un curso integral sobre el diseño y desarrollo de juegos.
- **Machine Learning de la A a la Z**: Un curso completo que cubre desde los conceptos básicos hasta aplicaciones avanzadas.
- **Fundamentos de Matemáticas para Machine Learning**: Cursos de matemáticas especializados para aspirantes a científicos de datos.

## Testimonios de Estudiantes
"Me encanta aprender en Frogames porque sus cursos son especializados y muy prácticos." - **Edwyn Mendoza**

"Siempre agradeceré las herramientas que Frogames me ha entregado." - **Javiera Constanza Vallejos Jara**

## Carreras y Oportunidades de Empleo
Frogames también busca talentos valiosos. Invitamos a personas apasionadas por la educación y que deseen ser parte de nuestro equipo a que se postulen para ser instructores o se conviertan en afiliados para compartir nuestras plataformas de capacitación.

## Certificaciones
Las certificaciones ofrecidas son robustas y están validadas por blockchain, asegurando su validez y reconocimiento en el campo profesional. Los estudiantes pueden fácilmente añadir estas certificaciones a su perfil de LinkedIn, aumentando su visibilidad y atractivo al empleador.

## Formación para Empresas
Ofrecemos programas de formación adaptados para empresas, ayudando a equipos a adquirir habilidades críticas en áreas como Power BI, Machine Learning y desarrollo de software, brindando así una ventaja competitiva en el mercado.

## Contacto
Para más información sobre nuestros cursos, oportunidades de empleo, o para empezar tu formación con nosotros, visita nuestro sitio web [Frogames Formación](https://www.frogamesformacion.com).

---

¡Únete a Frogames Formación y da el siguiente paso en tu educación y desarrollo profesional!

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("Anthropic", "https://anthropic.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://www.anthropic.com/company'}, {'type': 'Página de Carreras', 'url': 'https://www.anthropic.com/careers'}, {'type': 'Página de Claude', 'url': 'https://www.anthropic.com/claude'}, {'type': 'Página de Investigación', 'url': 'https://www.anthropic.com/research'}, {'type': 'Página de API', 'url': 'https://www.anthropic.com/api'}, {'type': 'Página de Precios', 'url': 'https://www.anthropic.com/pricing'}, {'type': 'Página de Clientes', 'url': 'https://www.anthropic.com/customers'}, {'type': 'Página de Noticias', 'url': 'https://www.anthropic.com/news'}]}


# Folleto de la Empresa: Anthropic

## Quiénes Somos
Anthropic es una empresa de investigación y seguridad de IA con sede en San Francisco. Nos dedicamos a construir sistemas de IA confiables, interpretables y dirigibles. Creemos en el impacto significativo que la IA tiene en el mundo y nos enfocamos en generar investigación sobre sus oportunidades y riesgos.

## Nuestra Misión
Nuestra misión es asegurar que la IA transforme la sociedad para bien. Nos esforzamos por desarrollar sistemas de vanguardia que sean seguros y beneficien a todos. La seguridad en la IA es una ciencia que abordamos de manera sistemática, compartiendo los conocimientos adquiridos con la comunidad global.

## Cultura de la Empresa
- **Colaboración**: Fomentamos un ambiente de confianza y colaboración, donde cada miembro del equipo contribuye al objetivo común.
- **Misión en Primer Lugar**: Cada acción que tomamos está guiada por nuestra misión de construir un futuro seguro y positivo para la humanidad.
- **Pragmatismo**: Valoramos la simplicidad y la eficacia en nuestras soluciones, buscando siempre las formas más efectivas de abordar los problemas complejos.

## Nuestro Equipo
El equipo de Anthropic está formado por investigadores, ingenieros, expertos en políticas y líderes operativos que provienen de diversos campos como la física, el aprendizaje automático y la política pública. Creemos que una diversidad de pensamientos y experiencias es fundamental para superar los desafíos que enfrenta la IA.

## Nuestros Clientes
Anthonycolaboramos con empresas, organizaciones sin fines de lucro y entidades políticas para promover la seguridad de la IA. La confianza y la integridad son la base de nuestras relaciones con los clientes, quienes utilizan Claude, nuestra plataforma de inteligencia artificial.

## Carreras en Anthropic
### Lo Que Ofrecemos
- **Beneficios de Salud y Bienestar**: Seguro médico, dental y de visión, apoyo para fertilidad, y 22 semanas de licencia parental.
- **Compensación Competitiva**: Sueldos competitivos y participación en acciones de la empresa, además de planes de jubilación.
- **Ambiente de Trabajo Inclusivo**: Estamos comprometidos con un entorno de alta confianza y apoyo.

### Proceso de Contratación
Buscamos candidatos con experiencia diversa. Valoramos las habilidades y el talento más que la experiencia previa en IA. Los entrevistados pueden acceder a recursos como Colab para demostrar sus habilidades y conocimiento técnico.

### Oportunidades de Desarrollo
Aunque no ofrecemos pasantías, animamos a una amplia variedad de candidatos a postularse, independientemente de su camino académico o profesional. Valoramos contribuciones previas en investigaciones o software de código abierto.

## Cursos y Capacitación para Futuro
Anthropic no ofrece cursos específicos, pero nuestro entorno de trabajo fomenta el aprendizaje continuo y la experimentación. Brindamos un estipendio educativo anual para apoyar el desarrollo profesional de nuestros empleados.

## Contáctanos
¿Te gustaría unirte a nosotros en nuestra misión de hacer que la IA sea segura y benéfica? Visita nuestro [sitio de carreras](#) para ver las oportunidades disponibles y cómo puedes contribuir.

---

Para más información sobre nuestros productos y servicios, visita nuestro sitio principal o síguenos en nuestras redes sociales. ¡Estamos ansiosos de conectar contigo! 

[Twitter](#) | [LinkedIn](#) | [YouTube](#)

**Anthropic PBC - Construyendo un futuro seguro con inteligencia artificial.**

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://huggingface.co'}, {'type': 'Página de Modelos', 'url': 'https://huggingface.co/models'}, {'type': 'Página de Datasets', 'url': 'https://huggingface.co/datasets'}, {'type': 'Página de Espacios', 'url': 'https://huggingface.co/spaces'}, {'type': 'Página de Precios', 'url': 'https://huggingface.co/pricing'}, {'type': 'Página de Carreras', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'Blog', 'url': 'https://huggingface.co/blog'}]}


# Hugging Face: La Comunidad de IA que Construye el Futuro

Bienvenido a Hugging Face, una plataforma líder donde la comunidad de aprendizaje automático colabora en la creación de modelos, conjuntos de datos y aplicaciones innovadoras. Con más de 50,000 organizaciones utilizando nuestros servicios, en Hugging Face estamos comprometidos con el avance de la inteligencia artificial y el aprendizaje automático.

## Cultura de la Empresa

En Hugging Face, promovemos una cultura de **colaboración abierta e innovación continua**. Valoramos el trabajo en equipo, la transparencia y el apoyo mutuo en la comunidad de IA, lo que nos impulsa a desarrollar herramientas que no solo sean útiles, sino que también sean accesibles para todos.

## Nuestros Clientes

Hugging Face es utilizado por una gran variedad de organizaciones, desde startups hasta gigantes de la tecnología. Algunos de nuestros principales clientes incluyen:

- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
  
Con estos aliados, hemos creado un entorno robusto para la investigación y desarrollo en inteligencia artificial.

## Oportunidades de Carrera

En Hugging Face, siempre buscamos talentos apasionados que deseen formar parte de nuestra misión de democratizar la inteligencia artificial. Ofrecemos una variedad de roles en desarrollo, investigación, marketing y atención al cliente. Si estás interesado en unirte a nuestro equipo innovador, visita nuestra sección de **[Empleos](#)** para explorar las oportunidades disponibles.

## Cursos y Packs para el Futuro

Para aquellos que buscan expandir sus habilidades, Hugging Face ofrece una variedad de recursos formativos, incluyendo tutoriales y documentación detallada sobre nuestros modelos y herramientas. Además, proporcionamos acceso a:

- **Modelos y Conjuntos de Datos**: Colabora y descubre más de 400,000 modelos y 100,000 conjuntos de datos.
- **Espacios**: Crea y comparte aplicaciones de IA en la comunidad.
- **Soluciones Empresariales**: Servicios de cómputo optimizados y soporte dedicado para empresas.

### Precios

- **Free**: Acceso a herramientas básicas y colaboración en modelos públicos.
- **PRO**: Suscripción de $9/mes que desbloquea funciones avanzadas y acceso prioritario.
- **Enterprise Hub**: Soluciones a medida para organizaciones que requieren nivel empresarial.

## Conclusión

Hugging Face es más que una plataforma; es una comunidad que persigue la próxima frontera de la inteligencia artificial. Únete a nosotros para innovar, aprender y construir el futuro. Para más información, visita nuestra web o regístrate hoy mismo para empezar a colaborar.

[Visita nuestro sitio](https://huggingface.co/)  | [Únete a nuestra comunidad en Discord](https://discord.gg/huggingface) 

*¡Esperamos verte pronto en Hugging Face!*

In [24]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

Links encontrados: {'links': [{'type': 'Página Acerca de', 'url': 'https://cursos.frogamesformacion.com/pages/instructores'}, {'type': 'Página de Certificaciones', 'url': 'https://cursos.frogamesformacion.com/pages/certificaciones'}, {'type': 'Página de Cursos', 'url': 'https://cursos.frogamesformacion.com/courses'}, {'type': 'Página de Rutas', 'url': 'https://cursos.frogamesformacion.com/pages/rutas'}, {'type': 'Página de Premios', 'url': 'https://cursos.frogamesformacion.com/pages/premios'}, {'type': 'Página para Empresas', 'url': 'https://cursos.frogamesformacion.com/pages/frogames-para-empresas'}, {'type': 'Página de Clientes', 'url': 'https://cursos.frogamesformacion.com/pages/nuestros-clientes'}, {'type': 'Página de Afiliados', 'url': 'https://cursos.frogamesformacion.com/pages/afiliados'}]}


# Frogames Formación

## Conviértete en un Experto

**Frogames Formación** es una plataforma educativa galardonada con el premio a la **Enseñanza online de datos y competencias digitales más innovadora de Europa 2023**. Nos especializamos en ofrecer una diversidad de cursos en áreas como:

- **Programación de Videojuegos**
- **Inteligencia Artificial**
- **Machine Learning**
- **Desarrollo de Apps**
- **Data Science**

## Cultura de la Empresa

En Frogames creemos en aprender mientras nos divertimos. Fomentamos un ambiente colaborativo donde nuestros estudiantes pueden crecer profesional y personalmente. Nuestro equipo de instructores expertos está comprometido a proporcionar una experiencia de aprendizaje excepcional, asegurando que cada curso sea práctico y relevante.

## Testimonios de Nuestros Estudiantes

Más de **500,000 estudiantes** han confiado en nosotros para desarrollar sus habilidades. Aquí algunos ejemplos de sus experiencias:

- "Realmente prácticos, muy avanzados en contenido." - **Edwyn Mendoza**
- "Me encanta ser parte de una comunidad donde se aprende, crece y también se disfruta." - **Javiera Constanza Vallejos Jara**
- "Estoy encantado de estar junto a esta comunidad de amigos." - **Eulogio Enamorado Pallares**

## Oportunidades de Empleo y Carrera

En Frogames, también ofrecemos oportunidades de carrera. Buscamos personas apasionadas por la educación y la tecnología para unirse a nuestro equipo. Además, si estás interesado en ejercer la enseñanza, puedes convertirte en afiliado y ser remunerado por cada venta que consigas.

## Cursos y Certificaciones

Ofrecemos múltiples **rutas de aprendizaje** y **certificados validados por blockchain**. Al finalizar un curso, obtendrás una certificación segura y verificada que puedes compartir en tus redes sociales y mejorar tu CV.

### Tipos de Cursos Disponibles

- **Fundamentos de Matemáticas para Machine Learning**
- **Curso Completo de Python**
- **Machine Learning de la A a la Z**
- **Desarrollo de Videojuegos con Unity**

## Beneficios de Unirte a Frogames

- **Suscripciones Anuales**: Acceso ilimitado a una variedad de cursos que se actualizan semanalmente.
- **Aprendizaje a tu Ritmo**: Rutas organizadas para que incrementes tu conocimiento paso a paso.
- **Comunidad Activa**: Una red de apoyo con otros estudiantes y profesionales en tu campo.

## ¡Empieza Hoy!

Conéctate con nosotros y descubre cómo nuestros cursos pueden impulsar tu desarrollo profesional y personal. Mejor aún, obtén acceso a un **curso gratuito** para iniciarte en el mundo del **trading algorítmico** con Python.

Para más información, visita nuestro [sitio web](https://www.frogamesformacion.com).

---

**Frogames Formación**: Donde la pasión por aprender transforma tu futuro. ¡Únete a nosotros!

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.